In [ ]:
# Install Transformers Library
!pip install transformers==3.0.0

In [ ]:
!pip install contractions
import contractions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Humor_project')

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast, BertTokenizer,TFBertModel
#import data_pre
#import Pickle as pickle
import nltk
nltk.download('punkt')
nltk.download('stopwords')

device = torch.device("cuda")
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

# Load Dataset

In [ ]:
df = pd.read_csv ('/content/drive/My Drive/Humor_project/train.csv')
df_1 = pd.read_csv('/content/drive/My Drive/Humor_project/new_dataset.csv')
#df= pd.concat([df, df_1], ignore_index=True, sort=False)
test_df = pd.read_csv('/content/drive/My Drive/Humor_project/dev.csv') 
#df.tail()
df_1

In [ ]:
df_1_text, df_2_text = train_test_split(df_1, 
                                                                    #random_state=2018, 
                                                                    test_size=0.5, 
                                                                   stratify=df_1['is_humor'])
#df_1_new = pd.DataFrame([df_1_text,df_1_labels],columns = ["text","is_humor"])
#df_1_new['text'] = (df_1_text)
#df_1_new['is_humor'] = df_1_labels
#df_1_new['text'] = pd.Series(df_1_text).values
#df= pd.concat([df, df_1_text], ignore_index=True, sort=False)
df_1_text

In [ ]:
#test_df=pd.read_csv ('/content/drive/My Drive/Humor_project/public_dev.csv')
#test_df=pd.read_csv ('/content/drive/My Drive/Humor_project/public_test.csv')
#test_df['humor_controversy'] = (test_df['humor_controversy'].fillna(0)).astype(int)


In [ ]:
#df = df.loc[df['offense_rating'] <=1 ]
#df = df.loc[df['humor_controversy'] <=1 ]

#df = df[pd.to_numeric(df.is_humor, errors='coerce').notnull()]
#df = df[pd.to_numeric(df.humor_rating, errors='coerce').notnull()]
#test_df = test_df.loc[test_df['is_humor']==1]

In [ ]:
test_df.head()

In [ ]:
df.shape

In [ ]:
def clean_humor(df):
  tempArr=[]
  for line in df:
    tmpL=line.lower()
    tempArr.append(tmpL)
  return tempArr

In [ ]:
#df['text'] = clean_humor(df['text'])

In [ ]:
# check class distribution
#df['humor_controversy'] = (df['humor_controversy'].fillna(0)).astype(int)
df['is_humor'].value_counts(normalize = True)


In [ ]:
#train_text = data_pre.data_processing(df[['text']])

In [ ]:
def data_clean(df1):
  df1['text'] = df1['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
  df1['text'] = [' '.join(map(str, l)) for l in df1['text']]
  df1['text'] = df1['text'].replace('\d+', '', regex=True)
  #df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")
  #test_df['text'] = test_df['text'].str.replace("[^a-zA-Z]", " ")
  #tokenized_doc = test_df['text'].apply(lambda x: x.split())
  #tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
  #detokenized_doc = [] 
  #for i in range(len(test_df)): 
  #  t = ' '.join(tokenized_doc[i]) 
  #  detokenized_doc.append(t) 
  #test_df['text'] = detokenized_doc
  return df1

In [ ]:
df = data_clean(df)

In [ ]:
#df1= df[['text','is_humor']]

In [ ]:
#database_text= pd.read_csv ('/content/drive/My Drive/Humor_project/database_text.csv')
#database = data_clean(database_text)
#database_humor= pd.read_csv ('/content/drive/My Drive/Humor_project/database_humor.csv')


In [ ]:
#df_all= pd.concat([df1, database], ignore_index=True, sort=False)
#df_all

# Split train dataset into train, validation and test sets

In [ ]:
#train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['is_humor'], 
                                                                    #random_state=2018, 
                                                                    #test_size=0.3, 
                                                                    #stratify=df['is_humor'])

#we will use temp_text and temp_labels to create validation and test set
#val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                               #random_state=2018, 
                                                               #test_size=0.5, 
                                                               # stratify=temp_labels)
#test_text = test_df['text']

In [ ]:
train_text, val_text, train_labels, val_labels = train_test_split(df['text'], df['is_humor'], 
                                                                    random_state=2018, 
                                                                    test_size=0.1, 
                                                                    stratify=df['is_humor'])

#we will use temp_text and temp_labels to create validation and test set
#test_text = test_df['text']
test_labels = test_df['is_humor']
test_text = test_df['text']

In [ ]:
#train_text, val_text, train_labels, val_labels = train_test_split(df['text'], df['is_humor'], 
#                                                                    random_state=2, 
#                                                                    test_size=0.1)
                                                                    #stratify=df['is_humor'])

#we will use temp_text and temp_labels to create validation and test set
#test_text = test_df['text']
#test_labels = test_df['humor_controversy']
#test_text = test_df['text']

In [ ]:
train_text

In [ ]:
#extra_data_text= database['text']
#extra_data_text[0]
#train_text[0]
#ac_train_text = np.concatenate([train_text,extra_data_text])
#ac_train_text.shape


In [ ]:
#extra_data_class= database_humor['is_humor']
#ac_train_labels = np.concatenate([train_labels,extra_data_class])
#ac_train_labels.shape
#ac_train_labels

In [ ]:
#database.dtypes

In [ ]:
#train_text.shape

# Import BERT Model and BERT Tokenizer

In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

# Tokenization

In [ ]:
# get length of all the messages in the train set
seq_len = [len(i.split()) for i in df['text']]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
max_seq_len = 30

In [ ]:
# tokenize and encode sequences in the training set
#tokenizer = tokenization.FullTokenizer(
#    train_test=train_test, do_lower_case=True)
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
#tokenizer = tokenization.FullTokenizer(
#   val_test=val_test, do_lower_case=True)
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
#tokenizer = tokenization.FullTokenizer(
#   test_test=test_test, do_lower_case=True)
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
tokens_train

# Convert Integer Sequences to Tensors

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)

train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)

val_sampler = SequentialSampler(val_data)

val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

# Define Model Architecture

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      self.dropout = nn.Dropout(0.1)      
      self.relu =  nn.ReLU()

      self.fc1 = nn.Linear(768,512)
      #self.fc2 = nn.Linear(512,256)
      self.fc2 = nn.Linear(512,2)
      #self.fc4 = nn.Linear(64,2)


      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
      _, cls_hs = self.bert(sent_id, attention_mask=mask)

      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      x = self.fc2(x)
      
      
      #x=self.relu(x)
      #x=self.dropout(x)
      #x = self.fc3(x)
      #x=self.relu(x)
      #x=self.dropout(x)
      #x=self.fc4
      x = self.softmax(x)

      return x

In [ ]:
model = BERT_Arch(bert)

model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-3)

# Find Class Weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

In [ ]:
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

cross_entropy  = nn.NLLLoss(weight=weights) 

epochs = 10

# Fine-Tune BERT

In [ ]:
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

# Start Model Training

In [ ]:
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

# Load Saved Model

In [ ]:
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

# Get Predictions for Test Data

In [ ]:
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [ ]:
preds = np.argmax(preds, axis = 1)
print (preds)
print(classification_report(test_y, preds))

In [ ]:
# confusion matrix
#pd.crosstab(test_y, preds)

In [ ]:
preds
test_text

In [ ]:
##count = 0
#for i in range(len(test_y)):
#  if (test_y[i] != preds[i]):
#    count+=1
#print (count)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
f1_score(test_y,preds),precision_score(test_y,preds),confusion_matrix(test_y,preds)

In [ ]:
test = pd.DataFrame()
test['index'] = test_df['id']
test['is_humor'] = preds


In [ ]:
#test_df

In [ ]:
from google.colab import files
test.to_csv('public_dev_bert.csv')
files.download('public_dev_bert.csv')